In [1]:
import json
import pandas as pd
import numpy as np
import os
import re
from dax_extract import read_data_model_schema
from pathlib import Path

class MetaData():
    
    def __init__(self,pbit_path,schema_path="No path"):
        self.path=Path(pbit_path)
        self.schema=schema_path
        self.pattern=r'[0-9]+'
        self.items=os.listdir()
        try:
            self.data=read_data_model_schema(self.path)
        except:
            try:
                a=open(self.schema,"r",encoding='utf-8')
                self.data=json.load(a)
            except:
                a=open(self.schema,"r",encoding='utf-16')
                self.data=json.load(a)  
                
    def extract_measures(self):
        all_mes_files=list(filter(lambda x:x.startswith('mea'),self.items))
        all_nums=list(map(lambda x:int(re.findall(self.pattern,x)[0]),all_mes_files))
        if len(all_nums)==0:
            m_path="measures0.xlsx"
        else:
            m_path="measures"+str(max(all_nums)+1)+".xlsx"
        
        tab_no=[]
        tab_name=[]
        mea_name=[]
        mea_exp=[]
        data_type=[]
        for i in range(len(self.data['model']['tables'])):
            if "measures" in self.data['model']['tables'][i]:
                for j in range(len(self.data['model']['tables'][i]['measures'])):
                    tab_no.append(i) 
                    mea_name.append(self.data['model']['tables'][i]['measures'][j]['name'])
                    tab_name.append(self.data['model']['tables'][i]['name'])
                    if 'expression' in self.data['model']['tables'][i]['measures'][j]:
                        if type(self.data['model']['tables'][i]['measures'][j]['expression'])==list:
                            mea_exp.append("".join(self.data['model']['tables'][i]['measures'][j]['expression']))
                        else:
                            mea_exp.append(self.data['model']['tables'][i]['measures'][j]['expression'])
                    else:
                        mea_exp.append("No expression")
                    if "dataType" in self.data['model']['tables'][i]['measures'][j]:
                        data_type.append(self.data['model']['tables'][i]['measures'][j]['dataType'])
                    else:
                        data_type.append("No Data Type")
            else:
                continue
            measures_df=pd.DataFrame({"table Number":tab_no,"table Name":tab_name,"Measure Name":mea_name,"Measure Expression":mea_exp,"Data Ttpe":data_type})
            measures_df.to_excel(m_path)
            
            
    def extract_relationships(self):
        all_rel_files=list(filter(lambda x:x.startswith('rel'),self.items))
        all_nums_rel=list(map(lambda x:int(re.findall(self.pattern,x)[0]),all_rel_files))
        if len(all_nums_rel)==0:
            r_path="relationships0.xlsx"
        else:
            r_path="relationships"+str(max(all_nums_rel)+1)+".xlsx"
        ft=[]
        fc=[]
        tt=[]
        tc=[]
        state=[]
                
        for i in self.data['model']['relationships']:
            ft.append(i['fromTable'])
            fc.append(i['fromColumn'])
            tt.append(i['toTable'])
            tc.append(i['toColumn'])
            if "state" in i:
                state.append(i['state'])
            else:
                state.append("No State")
    
        relationships=pd.DataFrame({"fromTable":ft,"fromColumn":fc,"toTable":tt,"toColumn":tc,"State":state})
        relationships.to_excel(r_path)

        
    def extract_calculated_columns(self):  
        all_col_files=list(filter(lambda x:x.startswith('col'),self.items))
        all_nums_col=list(map(lambda x:int(re.findall(self.pattern,x)[0]),all_col_files))
        if len(all_nums_col)==0:
            c_path="columns0.xlsx"
        else:
            c_path="columns"+str(max(all_nums_col)+1)+".xlsx"
        
        table_number=[]
        table_name=[]
        name=[]
        expression=[]
        for i in range(len(self.data['model']['tables'])):
            if "columns" in self.data['model']['tables'][i]:
                for j in range(len(self.data['model']['tables'][i]['columns'])):
                    if "type" in self.data['model']['tables'][i]['columns'][j]:
                        if self.data['model']['tables'][i]['columns'][j]['type']=='calculated':
                            table_number.append(i)
                            table_name.append(self.data['model']['tables'][i]['name'])
                            name.append(self.data['model']['tables'][i]['columns'][j]['name'])
                            if "expression" in self.data['model']['tables'][i]['columns'][j]:
                                if type(self.data['model']['tables'][i]['columns'][j]['expression'])==list:
                                    expression.append("".join(self.data['model']['tables'][i]['columns'][j]['expression']))
                                else:
                                    expression.append(self.data['model']['tables'][i]['columns'][j]['expression'])
                            else:
                                expression.append("No expression")
                        else:
                            continue
                    else:
                        continue

        col_df=pd.DataFrame({"table no":table_number,"Table Name":table_name,"name":name,"expression":expression})
        col_df.to_excel(c_path)
        
    def extract_partition(self):
        all_partitions_files=list(filter(lambda x:x.startswith('Par'),self.items))
        all_nums_partitions=list(map(lambda x:int(re.findall(self.pattern,x)[0]),all_partitions_files))
        if len(all_nums_partitions)==0:
            par_path="Partitions0.xlsx"
        else:
            par_path="Partitions"+str(max(all_nums_partitions)+1)+".xlsx"
            
        table_no=[]
        name=[]
        expression=[]
        for i in range(len(self.data['model']['tables'])):
            for j in range(len(self.data['model']['tables'][i]['partitions'])):
                table_no.append(i)
                if 'source' in self.data['model']['tables'][i]['partitions'][j]:
                    name.append(self.data['model']['tables'][i]['partitions'][j]['name'])
                    partition_name=[]
                    for k in name:
                        res=k.split("-")[0]
                        partition_name.append(res)
                    if self.data['model']['tables'][i]['partitions'][j]['source']['type']=='calculated':
                        if 'expression' in self.data['model']['tables'][i]['partitions'][j]['source']:
                            expression.append(self.data['model']['tables'][i]['partitions'][j]['source']['expression'])
                    else:
                        expression.append(self.data['model']['tables'][i]['partitions'][j]['source']['expression'])
        
        partitions=pd.DataFrame({'table_no':table_no,'partiton Name':partition_name,'expression':expression,})
        partitions.to_excel(par_path)
        
        
    def extract_hierarchy(self):
        all_hierarchy_files=list(filter(lambda x:x.startswith('hie'),self.items))
        all_nums_hierarchy=list(map(lambda x:int(re.findall(self.pattern,x)[0]),all_hierarchy_files))
        if len(all_nums_hierarchy)==0:
            h_path="hierarchy0.xlsx"
        else:
            h_path="hierarchy"+str(max(all_nums_hierarchy)+1)+".xlsx"
        table_no=[]
        hierarchy_levels_name=[]
        hierarchy_name=[]
        state=[]
        for i in range(len(self.data['model']['tables'])):
            if 'hierarchies' in self.data['model']['tables'][i]:
                for j in range(len(self.data['model']['tables'][i]['hierarchies'])):
                    if 'name' in self.data['model']['tables'][i]['hierarchies'][j]:
                        for k in range(len(self.data['model']['tables'][i]['hierarchies'][j]['levels'])):
                            state.append(self.data['model']['tables'][i]['hierarchies'][j]['state'])
                            hierarchy_name.append(self.data['model']['tables'][i]['hierarchies'][j]['name'])
                            table_no.append(i)
                            hierarchy_levels_name.append(self.data['model']['tables'][i]['hierarchies'][j]['levels'][k]['name'])
        hierarchy=pd.DataFrame({'table_no':table_no,'hierarchy_name':hierarchy_name,'levels_name':hierarchy_levels_name,'state':state})
        hierarchy.to_excel(h_path)   

In [8]:
a=MetaData("anjknfk","Weave.txt")

In [9]:
a.extract_measures()

In [10]:
a.extract_relationships()

In [11]:
a.extract_calculated_columns()

In [12]:
a.extract_partition()

In [2]:
b=MetaData("skbdkf","BFSI.txt")

In [3]:
b.extract_measures()

In [4]:
b.extract_relationships()

In [5]:
b.extract_calculated_columns()

In [6]:
b.extract_partition()

In [7]:
b.extract_hierarchy()

In [13]:
a.extract_hierarchy()